## related package

```python
pip install pdfminer.six
pip install unstructured_pytesseract
pip install unstructured_inference
```

[--->源地址<---](https://github.com/CodingLucasLi/GPT_Resume_analysing)

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader

import openai
import time
import datetime

## input data: JD, Option, Resumes

In [3]:
JD = """
1、全日制统招本科及以上学历，计算机、应用数学、统计学等专业，具有4年及以上数据开发及大数据平台建设经验；
2、精通主流数据库系统（如MySQL/SQL SERVER/Oracle/PostgralSQL等），有丰富的 SQL 编程经验及 ETL 流程设计开发经验，有MaxCompute /Hive/Spark SQL等相关开发经验;
3、熟悉数据仓库各类建模理论以及数据仓库数据层级关系，具备大型数据仓库逻辑模型和物理模型设计经验； 
4、熟练使用Python/Java/Scala/Shell等开发语言中的一种或多种；
6、熟悉Hadoop生态体系，了解HDFS、Hive、Spark、sqoop、HBase、MapReduce、Flume、kafka等开源项目原理和开发使用；
"""

In [4]:
options = ["姓名", "联系号码", "性别", "年龄", "工作年数（数字）", 
           "最高学历", "本科学校名称", "硕士学校名称", "是否在职", 
           "当前职务", "历史任职公司列表", "技术能力", "经验程度", "管理能力"]

In [5]:
input_dir = './data/resume/'

In [6]:
resumes = []
for input_path in os.listdir(input_dir):
    if not input_path.endswith('.txt'):
        continue
    
    with open(os.path.join(input_dir, input_path)) as f:
        resumes.append([input_path, f.read()])

In [7]:
resumes.__len__()

3

In [2]:
# print(resumes[1])

## batch-process-example

In [8]:
def ask_openAI(question):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=question,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0,
    )
    return response.choices[0].text.strip()

In [3]:
# for input_path, resume in resumes:
    
#     start = datetime.datetime.now()
#     print('processing', input_path, start)
    
#     # step 1: resume key-point extracting
#     text_splitter = CharacterTextSplitter(separator='\n\n', chunk_size=600, chunk_overlap=100)
#     chunks = text_splitter.split_text(resume)
#     embeddings = OpenAIEmbeddings()
#     knowledge_base = FAISS.from_texts(chunks, embeddings)
    
#     rows = []
#     for i, option in enumerate(options):
#         question = f"这个应聘者的{option}是什么，请精简返回答案，最多不超过250字，如果查找不到，则返回'未提供'"
#         docs = knowledge_base.similarity_search(question)
#         llm = OpenAI(temperature=0.3, model_name="text-davinci-003", max_tokens="100")
#         chain = load_qa_chain(llm, chain_type="stuff")
#         response = chain.run(input_documents=docs, question=question )
#         item = {'option': option, 'value': response }
#         print(i, item)
#         rows.append(item)
        
#         # reduce the concurrent
#         time.sleep(20)
        
#     df_candidate = pd.DataFrame(rows)
#     context = df_candidate.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x).to_string(index=False)
    
#     # step 2: summary analyze
#     summary_question = f"职位要求是：{{{JD}}}" + f"简历概要是：{{{context}}}" + "，请直接返回该应聘岗位候选人匹配度概要（控制在200字以内）;'"
#     summary = ask_openAI(summary_question)
#     df_candidate.loc[len(df_candidate)] = ['综合概要', summary]
    
#     # step 3: scoring
#     extra_info = "打分要求：岗位匹配+10分, 国内top10大学+3分，985大学+2分，211大学+1分，头部企业经历+2分，知名企业+1分，海外背景+3分，外企背景+1分。 "
#     score_question = f"职位要求是：{{{JD}}}" + f"简历概要是：{{{df_candidate.to_string(index=False)}}}" + "，请直接返回该应聘岗位候选人的匹配分数（0-100），请精确打分以方便其他候选人对比排序，'" + extra_info
#     score = ask_openAI(score_question)
#     df_candidate.loc[len(df_candidate)] = ['匹配得分', score]
#     df_candidate.to_csv(f'./reports/report_{input_path}.csv', encoding='utf-8-sig')
    
#     end = datetime.datetime.now()
#     print('complete', input_path, (end - start).total_seconds())
    

## single resume processing Example 

### resume key-point extracting.

In [8]:
text_splitter = CharacterTextSplitter(separator='\n\n', chunk_size=600, chunk_overlap=100)

In [9]:
chunks = text_splitter.split_text(resumes[1])

In [10]:
chunks.__len__()

48

In [8]:
# chunks[0]

In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [1]:
# rows = []
# for i, option in enumerate(options):
#     question = f"这个应聘者的{option}是什么，请精简返回答案，最多不超过250字，如果查找不到，则返回'未提供'"
#     docs = knowledge_base.similarity_search(question)
#     llm = OpenAI(temperature=0.3, model_name="text-davinci-003", max_tokens="2000")
#     chain = load_qa_chain(llm, chain_type="stuff")
#     response = chain.run(input_documents=docs, question=question )
#     item = {'option': option, 'value': response }
#     print(i, item)
#     rows.append(item)
    
#     # reduce the concurrent
#     time.sleep(15)

```
0 {'option': '姓名', 'value': ' 夏天'}
1 {'option': '联系号码', 'value': ' 联系号码：13153518619'}
2 {'option': '性别', 'value': ' 男'}
3 {'option': '年龄', 'value': ' 45岁'}
4 {'option': '工作年数（数字）', 'value': ' 23年'}
5 {'option': '最高学历', 'value': ' 夏天的最高学历是硕士。'}
6 {'option': '本科学校名称', 'value': ' 江苏师范大学'}
```

In [16]:
df_candidate = pd.DataFrame(rows)

In [17]:
df_candidate

,option,value
0,姓名,夏天
1,联系号码,联系号码：13153518619
2,性别,男
3,年龄,45岁
4,工作年数（数字）,23年
5,最高学历,夏天的最高学历是硕士。
6,本科学校名称,江苏师范大学
7,硕士学校名称,江苏师范大学
8,是否在职,在职
9,当前职务,夏天目前的职务是IT技术/研发总监，负责公司软件研发部研发工作，售前支持，客户需求沟通，技...


In [22]:
context = df_candidate.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x).to_string(index=False)

>  回头看下context的质量

### summary analyze

In [4]:
# summary_question = f"职位要求是：{{{JD}}}" + f"简历概要是：{{{context}}}" + "，请直接返回该应聘岗位候选人匹配度概要（控制在200字以内）;'"
# print(summary_question)

In [27]:
response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=summary_question,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0,
    )

In [5]:
# response

In [6]:
# summary = response.choices[0].text.strip()
# print(summary)

In [31]:
df_candidate.loc[len(df_candidate)] = ['综合概要', summary]

In [33]:
extra_info = "打分要求：国内top10大学+3分，985大学+2分，211大学+1分，头部企业经历+2分，知名企业+1分，海外背景+3分，外企背景+1分。 "

In [7]:
# score_question = f"职位要求是：{{{JD}}}" + f"简历概要是：{{{df_candidate.to_string(index=False)[:-500]}}}" + "，请直接返回该应聘岗位候选人的匹配分数（0-100），请精确打分以方便其他候选人对比排序，'" + extra_info
# print(score_question)

In [42]:
response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=score_question,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0,
    )

In [43]:
response.choices[0].text.strip()

'90分'